In [ ]:
import analytics.api as ga
import analytics.sheets_api as sheets
import analytics.sheets_elements as elements
import pandas as pd
from constants import *

In [ ]:
%env HCA_ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../do_not_commit_ga4_credentials.json

In [ ]:
ga_authentication, drive_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    port=OAUTH_PORT
)

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
}

hca_portal_params = {
    **default_params,
    "base_dimension_filter": HCA_BROWSER_EXCLUDE_FILTER,
    "property": HCA_PORTAL_ID,
}



In [ ]:
df_outbound = elements.get_outbound_links_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)

In [ ]:
dict_spreadsheet = {
    "Outbound Links": df_outbound,
}
sheets.fill_spreadsheet_with_df_dict(
    sheets.create_sheet_in_folder(
        drive_authentication,
        SHEET_NAME,
        PARENT_FOLDER_NAME,
        override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
    ),
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Outbound Links": {
            "Total Clicks Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        }
    }
)